In [12]:
import pandas as pd
from transformers import pipeline
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertConfig
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import numpy as np
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers import evaluation

In [ ]:

path = '/content/drive/MyDrive/A3_task1_data_files'

task1_train_data =  pd.read_csv(path + '/train.csv', sep = "\t")
task1_train_data.dropna(inplace=True)
task1_train_data.reset_index(drop=True, inplace=True)

task1_val_data =pd.read_csv(path +'/dev.csv', sep = "\t")
task1_val_data = task1_val_data.rename(columns = {'setence1':'sentence1'})
task1_val_data.dropna(inplace=True)
task1_val_data.reset_index(drop=True, inplace=True)


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

,score,sentence1,sentence2
0,5.00,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.
1,4.75,A young child is riding a horse.,A child is riding a horse.
2,5.00,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.
3,2.40,A woman is playing the guitar.,A man is playing guitar.
4,2.75,A woman is playing the flute.,A man is playing a flute.


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = SentenceTransformer('distilbert-base-nli-mean-tokens').to(device)
train_examples = []
for i in range(len(task1_train_data)):
    train_examples.append(InputExample(texts=[task1_train_data.iloc[i]['sentence1'], task1_train_data.iloc[i]['sentence2']], label=float(task1_train_data.iloc[i]['score']/5.0)))
    
train_batch_size = 32
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

val_examples = []
for i in range(len(task1_val_data)):
    val_examples.append(InputExample(texts=[task1_val_data.iloc[i]['sentence1'], task1_val_data.iloc[i]['sentence2']], label=task1_val_data.iloc[i]['score']/5.0))
val_batch_size = 32
val_dataloader = DataLoader(val_examples, shuffle=True, batch_size=val_batch_size)


In [ ]:


evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name='val')
# Fine-tune the model with the custom callback
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=10,
          warmup_steps=100,
          output_path='fine-tuned-model')

In [ ]:
model.save("/content/drive/MyDrive/model_1C")

In [ ]:
values = []
for i in range(len(task1_val_data)):
    # print("Calculating for: ", i)
    embed1 = model.encode(task1_val_data.iloc[i]['sentence1'])
    embed2 = model.encode(task1_val_data.iloc[i]['sentence2'])
    similarity = np.dot(embed1, embed2)/(np.linalg.norm(embed1)*np.linalg.norm(embed2))
    values.append(similarity*5)
pearson = np.corrcoef(values, task1_val_data['score'])
print("Pearson Correlation: ", pearson[0][1])